<h1 align="center"> Principals of Biomedical Engineering </h1>
<h3 align="center"> Dr. M. Jahed </h3>
<br>
<h2 align="center"> EMG Classificatioh </h2>
<br>
<h4 align="center">Mohammad Hossein Shafieizadegan</h4>
<h4 align="center">Navid Bagheri Shouraki</h4>

# Load Data

Load the dataset, whether our own acquired data or online free available dataset

# Signal Preprocessing

The following steps must be done:
- Bandpass Filtering
- Notch filter
- Z-score

# Signal Windowing

# Feature Extraction

Recommended features:
- Mean absolute value (MAV)
- Standard deviation (STD)
- Variance (VAR)
- Waveform length (WL)
- Zero crossing (ZC)
- Root mean square (RMS)
- Number of peaks (NP)
- Slope sign change (SSC)
- Correlation coefficient (Cor)
- Integrated absolute value (IAV)

# Classification and model training